# Test general

In [41]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy

^C


## YOLOv3


https://github.com/ultralytics/yolov3

In [4]:
import requests

# Descargar los archivos de YOLOv4-tiny
yolov4_weights_url = "https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4-tiny.weights"
yolov4_cfg_url = "https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg"
coco_names_url = "https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true"

# Función para descargar archivos
def download_file(url, filename):
    with open(filename, "wb") as f:
        response = requests.get(url)
        f.write(response.content)

# Descargar los archivos
download_file(yolov4_weights_url, "yolov4-tiny.weights")
download_file(yolov4_cfg_url, "yolov4-tiny.cfg")
download_file(coco_names_url, "coco.names")



## Script

In [5]:
import cv2
import numpy as np
import threading

In [8]:
weights = "C:/Users/mateo/OneDrive/Escritorio/Projects/DeteccionDeObjetos/objectdetection/models/yolov4-tiny.weights"
config = "C:/Users/mateo/OneDrive/Escritorio/Projects/DeteccionDeObjetos/objectdetection/models/yolov4-tiny.cfg"
coco = "C:/Users/mateo/OneDrive/Escritorio/Projects/DeteccionDeObjetos/objectdetection/models/coco.names"

# Cargar la red YOLOv4-tiny
net = cv2.dnn.readNet(weights, config)
classes = []
with open(coco, "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


In [19]:

# Clases que queremos detectar
classes_to_detect = ["bicycle", "car", "motorbike", "bus", "train", "traffic light", "stop sign"]


In [26]:
# Función para detección de objetos
def detect_objects(img):
    height, width, channels = img.shape

    # Preprocesamiento de la imagen
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Información de detección
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] in classes_to_detect:
                # Objeto detectado
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Coordenadas de la caja
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Supresión no máxima para eliminar cajas superpuestas
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Dibujar cajas y etiquetas en la imagen
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes_to_detect), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[classes_to_detect.index(label)]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 2)

    return img

In [25]:
# Captura de video desde la cámara web
cap = cv2.VideoCapture("C:/Users/mateo/OneDrive/Escritorio/Projects/DeteccionDeObjetos/objectdetection/data/raw/rotonda.mp4")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # Ancho
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # Alto
while True:
    # Leer el fotograma de la cámara
    ret, frame = cap.read()
    # Si no se puede leer el fotograma, salir del bucle
    if not ret:
        break

    # Detectar objetos en el fotograma
    detected_frame = detect_objects(frame)

    # Mostrar el fotograma con objetos detectados
    cv2.imshow('Object Detection', detected_frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()